In [2]:
import torch
import pandas as pd

data = pd.read_csv('./datasets/iris_extended.csv')
data.head()

,species,elevation,soil_type,sepal_length,sepal_width,petal_length,petal_width,sepal_area,petal_area,sepal_aspect_ratio,...,sepal_to_petal_length_ratio,sepal_to_petal_width_ratio,sepal_petal_length_diff,sepal_petal_width_diff,petal_curvature_mm,petal_texture_trichomes_per_mm2,leaf_area_cm2,sepal_area_sqrt,petal_area_sqrt,area_ratios
0,setosa,161.8,sandy,5.16,3.41,1.64,0.26,17.5956,0.4264,1.513196,...,3.146341,13.115385,3.52,3.15,5.33,18.33,53.21,4.194711,0.652993,41.265478
1,setosa,291.4,clay,5.48,4.05,1.53,0.37,22.1940,0.5661,1.353086,...,3.581699,10.945946,3.95,3.68,5.90,20.45,52.53,4.711051,0.752396,39.205087
2,setosa,144.3,sandy,5.10,2.80,1.47,0.38,14.2800,0.5586,1.821429,...,3.469388,7.368421,3.63,2.42,5.66,24.62,50.25,3.778889,0.747395,25.563910
3,setosa,114.6,clay,4.64,3.44,1.53,0.17,15.9616,0.2601,1.348837,...,3.032680,20.235294,3.11,3.27,4.51,22.91,50.85,3.995197,0.510000,61.367166
4,setosa,110.9,loamy,4.85,2.87,1.23,0.26,13.9195,0.3198,1.689895,...,3.943089,11.038462,3.62,2.61,4.03,21.56,40.57,3.730885,0.565509,43.525641


In [3]:
class IrisDataset(torch.utils.data.Dataset):
  def __init__(self):
    self.labels = pd.get_dummies(data['species']) \
      .astype('float32').to_numpy()

    self.data = pd.get_dummies(data, columns=['soil_type'], dtype='float32') \
        .drop('species', axis=1) \
        .astype('float32').to_numpy()
    
  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx], self.labels[idx]

dataset = IrisDataset()
train, test = torch.utils.data.random_split(dataset, [0.7, 0.3])

train_loader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=True)

print(dataset.data)

[[161.8    5.16   3.41 ...   0.     0.     1.  ]
 [291.4    5.48   4.05 ...   1.     0.     0.  ]
 [144.3    5.1    2.8  ...   0.     0.     1.  ]
 ...
 [ 73.6    6.79   3.25 ...   1.     0.     0.  ]
 [239.6    6.38   2.24 ...   0.     0.     1.  ]
 [201.5    5.16   3.2  ...   0.     1.     0.  ]]


In [4]:
class IrisNeuralNetwork(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.flatten = torch.nn.Flatten()
    self.linear_relu_stack = torch.nn.Sequential(
      torch.nn.Linear(22, 8),
      torch.nn.ReLU(),
      torch.nn.Linear(8, 8),
      torch.nn.ReLU(),
      torch.nn.Linear(8, 3),
      torch.nn.Softmax(dim=1)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits
  
model = IrisNeuralNetwork().to("cpu")
print(model)

IrisNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=22, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=3, bias=True)
    (5): Softmax(dim=1)
  )
)


In [5]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1000):
  for i, (inputs, targets) in enumerate(train_loader):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
      print('Epoch: %d, Loss: %.3f' % (epoch, loss.item()))

Epoch: 0, Loss: 0.276
Epoch: 0, Loss: 0.221
Epoch: 0, Loss: 0.194
Epoch: 0, Loss: 0.179
Epoch: 0, Loss: 0.271
Epoch: 0, Loss: 0.263
Epoch: 0, Loss: 0.194
Epoch: 0, Loss: 0.355
Epoch: 0, Loss: 0.341
Epoch: 0, Loss: 0.122
Epoch: 0, Loss: 0.115
Epoch: 0, Loss: 0.124
Epoch: 0, Loss: 0.133
Epoch: 0, Loss: 0.116
Epoch: 0, Loss: 0.090
Epoch: 0, Loss: 0.111
Epoch: 0, Loss: 0.148
Epoch: 0, Loss: 0.134
Epoch: 0, Loss: 0.104
Epoch: 0, Loss: 0.084
Epoch: 0, Loss: 0.122
Epoch: 0, Loss: 0.114
Epoch: 0, Loss: 0.111
Epoch: 0, Loss: 0.095
Epoch: 0, Loss: 0.089
Epoch: 0, Loss: 0.083
Epoch: 0, Loss: 0.102
Epoch: 100, Loss: 0.001
Epoch: 100, Loss: 0.006
Epoch: 100, Loss: 0.000
Epoch: 100, Loss: 0.009
Epoch: 100, Loss: 0.000
Epoch: 100, Loss: 0.003
Epoch: 100, Loss: 0.028
Epoch: 100, Loss: 0.010
Epoch: 100, Loss: 0.010
Epoch: 100, Loss: 0.000
Epoch: 100, Loss: 0.010
Epoch: 100, Loss: 0.018
Epoch: 100, Loss: 0.016
Epoch: 100, Loss: 0.002
Epoch: 100, Loss: 0.001
Epoch: 100, Loss: 0.000
Epoch: 100, Loss: 0.02

In [9]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for inputs, targets in test_loader:
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    target_class = torch.argmax(targets, dim=1)
    
    total += targets.size(0)
    correct += (predicted == target_class).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 98.89%
